I'm a totally new bee in Kaggle and this is my first competition, got ~1114 in public leaderboard using pure Xgboost in raw features with 5-fold cross_validation. I have learned a lot from kaggle forums and a friend.   I want to share it with all starters. Hope it will be helpful for someone. If there are any mistakes, please do not hesitate to let me know.

Due to this NoteBook cannot write files, so If you want to run my code, copy and paste them to your local workspace.  

###Step 1 

Transform letters of categories in train.csv and test.csv into numbers for computation, using script bellow. you will get two files named train_num.csv and test_num.csv

    import pandas as pd
    import numpy as np
    
    np.random.seed(1234)
    def LetterToNumber(data,dest_file):
        for c in data.columns:
            if 'cat' in c:
                print c
                data[c]=data[c].astype("category")
                data[c].cat.categories=range(len(set(data[c].values)))
        data.to_csv(dest_file, index=False)
    
    
    train = pd.read_csv("./data/train.csv")
    train_dest = "./data/train_num.csv"
    LetterToNumber(train,train_dest)
    test = pd.read_csv("./data/test.csv")
    test_dest = "./data/test_num.csv"
    LetterToNumber(test,test_dest)



###Step 2

randomly split train data into five parts in preparation for cross_validation. 


    #split_cv.py
    
    import numpy as np
    
    np.random.seed(1234)
    
    ids=[]
    for line in open("./data/cut_cv"):
        ids.append(int(line))
    train=[0]*5
    test=[0]*5
    for i in range(5):
        train[i]=open("./data/train_cv_"+str(i)+".csv","w")
        test[i]=open("./data/test_cv_"+str(i)+".csv","w")
    for j,line in enumerate(open("./data/train_num.csv")):
        if j==0:
            for i in range(5):
                train[i].write(line)
                test[i].write(line)
            continue
        else:
            for id in range(5):
                if id != ids[j-1]:
                    train[id].write(line)
            test[ids[j-1]].write(line)

### Step 3

using Xgboost to train and predict. 

we will get five predictions on test_num.csv.

    #train_predict.py
    
    import xgboost
    import pandas as pd
    import numpy as np
    import datetime
    from sklearn.metrics import mean_absolute_error
    
    np.random.seed(1234)
    
    def logregobj(preds, dtrain):
        labels = dtrain.get_label()
        con = 2
        x = preds-labels
        grad = con*x / (np.abs(x)+con)
        hess = con**2 / (np.abs(x)+con)**2
        return grad, hess
    
    def xg_eval_mae(yhat, dtrain):
        y = dtrain.get_label()
        return 'mae', mean_absolute_error(np.exp(y)-shift,
                                          np.exp(yhat)-shift)
    
    
    for CV in range(5):
        CV = str(CV)
        print 'loading...',CV
        train=pd.read_csv("./data/train_cv_"+CV+".csv")
        test=pd.read_csv("./data/test_cv_"+CV+".csv")
        shift=200
    
        print 'droping...'
        train_X=train.drop(['id','loss'],axis=1)
        train_Y=np.log(train['loss']+shift)
        test_X=test.drop(['id','loss'],axis=1)
        test_Y=np.log(test['loss']+shift)
    
        train=xgboost.DMatrix(train_X,label=train_Y)
        test=xgboost.DMatrix(test_X,label=test_Y)
        #8 12
        param = {'max_depth':8 , 'gamma': 0, 'silent': 0, 'boost': 'gbtree', 'objective': 'reg:linear',
                 'alpha': 1, 'subsample': 0.86, "min_child_weight": 50,
                 "colsample_bytree": 0.32, 'colsample_bylevel': 1}
    
    
        param['nthread'] = 1
        num_round = 200000
    
        watchlist = [(train,'train'),(test,'test')]
    
        print "trainning..."
        bst = xgboost.train(param.items(),train, num_round,watchlist,learning_rates=[0.02]*num_round,obj=logregobj,
                            feval=xg_eval_mae,maximize=False,early_stopping_rounds=500)
    
        for i,line in enumerate(sorted(bst.get_score().iteritems(),key=lambda d:d[1], reverse=True)):
            if i>30:
                break
            print line
    
        print 'loading...'
        raw_test=pd.read_csv("./data/test_num.csv")
        test=xgboost.DMatrix(raw_test.drop(['id'],axis=1))
    
        print 'predicting...'
        result=np.exp(bst.predict(test))-200
        now = datetime.datetime.now()
        pd.DataFrame({'id':raw_test['id'].values,'loss':result}).to_csv('./result/cv_cls2/'+
                                                                        now.strftime("%Y-%m-%d-%H-%M")+
                                                                        "_"+CV+".csv",index=False)

### Step 4

The last thing is to **average out** above five results on test_num.csv and get the final prediction.

    #merge_result.py
    
    import os
    import datetime
    dir=os.listdir("./result/cv_cls2/")
    ID=[]
    temp = []
    ct=[]
    for i,file in enumerate(dir):
        file="./result/cv_cls2/"+file
        print file
        for j,line in enumerate(open(file)):
            if j==0:
                continue
            if i==0:
                ID.append(line.split(",")[0])
                temp.append(float(line.split(",")[1]))
                ct.append(1)
            else:
                temp[j-1]+=float(line.split(",")[1])
                ct[j-1]+=1
    now = datetime.datetime.now()
    
    path = './result/' + str(now.strftime("%Y-%m-%d-%H-%M")) + '_cls2.csv'
    tow=open(path,"w")
    tow.write("id,loss\n")
    for i,line in enumerate(temp):
        r=temp[i]/ct[i]
        tow.write(ID[i]+","+str(r)+"\n")
    tow.close()